In [ ]:
from common import *
from snobedo.snotel import CsvParser

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

# Comparison between SNOTEL and iSnobal variables

In [ ]:
year = 2021
water_year = f"wy{year}"
time=slice(f"{year -1}-10-01", f"{year}-07-31")

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

## SNOTEL

In [ ]:
schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv',
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## iSnobal 

In [ ]:
wy_snow = xr.open_mfdataset(
    (SNOBAL_DIR / f'{water_year}' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

butte_snobal = wy_snow.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
schofield_snobal = wy_snow.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
taylor_snobal = wy_snow.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()

In [ ]:
def max_swe_date(snotel_data):
    return snotel_data.loc[snotel_data['SWE(mm)'].idxmax()].name

In [ ]:
plot_data = {
    'Butte': {
        'snobal': butte_snobal,
        'snotel': butte_snotel_csv,
        'color_snobal': 'sandybrown',
        'color_snotel': 'skyblue',
        'marker_aso': '^',
        'max_swe_snotel': max_swe_date(butte_snotel_csv)
    },
    'Schofield Pass': {
        'snobal': schofield_snobal,
        'snotel': schofield_snotel_csv,
        'color_snobal': 'goldenrod',
        'color_snotel': 'steelblue',
        'marker_aso': '^',
        'max_swe_snotel': max_swe_date(schofield_snotel_csv)
    },
    'Upper Taylor': {
        'snobal': taylor_snobal,
        'snotel': taylor_snotel_csv,
        'color_snobal': 'bisque',
        'color_snotel': 'royalblue',
        'marker_aso': '^',
        'max_swe_snotel': max_swe_date(taylor_snotel_csv)
    },
}

### ASO Snow Depth 

In [ ]:
for site in plot_data.keys():
    plot_data[site]['ASO'] = pd.read_csv(DATA_DIR / f'ASO-data/SNOTEL-sites/{site}.csv', delimiter=',', header=None, parse_dates=[0], index_col=0)

## Compare SNOTEL to iSnobal

In [ ]:
from matplotlib.offsetbox import AnchoredText
import matplotlib.lines as lines
import math

In [ ]:
figure_opts = dict(figsize=(8,4.5), dpi=300,)
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=11, freq='MS')
text_props = dict(facecolor='whitesmoke', alpha=0.5, pad=.5, boxstyle='round')

xTicks = mdates.DateFormatter('%d-%b')

In [ ]:
def snobal_max_swe(site):
    return site.where(site.specific_mass == site.specific_mass.max(), drop=True).time[0].values


def swe_numbers(site, max_snobal, min_snobal):
    snobal_min = int(min_snobal['specific_mass'].sum())
    snobal_max = int(max_snobal['specific_mass'].sum())
    
    max_swe_date_snotel = plot_data[site]['max_swe_snotel'].date()
    max_swe_date_snobal = snobal_max_swe(plot_data[site]['snobal'])
    swe_date_diff = (max_swe_date_snobal - np.datetime64(max_swe_date_snotel)).astype('timedelta64[D]')
    
    snotel_sum = int(plot_data[site]['snotel']['SWE(mm)'].sum())
    
    legend = {}
    legend['Snotel'] = {}
    legend['Snotel']['total  '] = f'{snotel_sum} mm'
    legend['Snotel']['max SWE'] = str(max_swe_date_snotel)
    legend['Snobal'] = {}
    legend['Snobal']['max SWE'] = np.datetime_as_string(max_swe_date_snobal, unit='D')
    legend['Snobal']['  diff '] = str(swe_date_diff)
    legend['Snobal'][''] = ''
    legend['Snobal']['max total   '] = f'{snobal_max} mm'
    legend['Snobal']['min total   '] = f'{snobal_min} mm'
    legend['Snobal']['% Snotel max'] = f'{snobal_max/snotel_sum:.2%}'
    legend['Snobal']['% Snotel min'] = f'{snobal_min/snotel_sum:.2%}'

    return legend


def snotel_snobal_depth_diff(site):
    mean_snobal = plot_data[site]['snobal']['thickness'].coarsen(**COARSEN_OPTS).mean()

         
def depth_numbers(site, max_snobal, min_snobal):
    melt_start = plot_data[site]['max_swe_snotel']
    snobal_depths = min_snobal['thickness'].squeeze('x').squeeze('y').to_pandas()
    min_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]
    
    snobal_depths = max_snobal['thickness'].squeeze('x').squeeze('y').to_pandas()
    max_date = snobal_depths[(snobal_depths == 0) & (snobal_depths.index > melt_start)].index[0]

    snotel_depth = plot_data[site]['snotel']['Depth(m)']
    snotel_date = snotel_depth[(snotel_depth == 0) & (snotel_depth.index > melt_start)].index[0]

    legend = {}
    legend['Snotel melt-out'] = snotel_date.strftime(LEGEND_DATE)
    legend['Snobal Min     '] = min_date.strftime('%Y-%m-%d')
    legend['Snobal Max     '] = max_date.strftime(LEGEND_DATE)
    legend['Snobal Min diff'] = f'{(min_date - snotel_date).days} days'
    legend['Snobal Max diff'] = f'{(max_date - snotel_date).days} days'
    
    return legend


def print_hash(hash):
    import json
    import re
    print(re.sub(r'[{,},"]*', '', json.dumps(hash, indent=4)))

In [ ]:
def aso_scatter(ax, site_name, max_snobal, min_snobal, mean_snobal):
    df = plot_data[site_name]['ASO']
    dates = df[df.index.year == year].index.unique().values
    legend_entry = 0
    stats = {}
    
    for day in dates:
        if legend_entry == 0:
            label_entry=dict(label='ASO')
        else:
            label_entry=dict()

        max_depth = df[df.index == day].max().values[0]
        min_depth = df[df.index == day].min().values[0]
        mean_depth = df[df.index == day].mean().values[0]
        
        day_key = np.datetime_as_string(day, unit='D')
        stats[day_key] = {}
        stats[day_key]['ASO max    '] = f'{max_depth:.2f} m'
        stats[day_key]['ASO min    '] = f'{min_depth:.2f} m'
        snotel_value = plot_data[site_name]['snotel'][plot_data[site]['snotel'].index == day]['Depth(m)'].values[0]
        stats[day_key]['SNOTEL     '] = f'{snotel_value:.2f} m'
        snobal_value = max_snobal.where(max_snobal.time.dt.floor('D') == day, drop=True).squeeze(['x', 'y'])['thickness'].values[0]
        stats[day_key]['iSnobal max'] = f'{snobal_value:.2f} m'
        snobal_value = min_snobal.where(min_snobal.time.dt.floor('D') == day, drop=True).squeeze(['x', 'y'])['thickness'].values[0]
        stats[day_key]['iSnobal min'] = f'{snobal_value:.2f} m'
        snobal_value = mean_snobal.where(mean_snobal.time.dt.floor('D') == day, drop=True).squeeze(['x', 'y'])['thickness'].values[0]
        stats[day_key]['iSnobal mean diff'] = f'{snobal_value - mean_depth:.2f} m'
        
        if max_depth >= 0 and min_depth >= 0:
            ax.scatter(
                day, 
                max_depth, 
                marker='_', 
                color='black',
                **label_entry
            )
            ax.scatter(
                day, 
                min_depth, 
                marker='_', 
                color='black',
            )
            ax.plot([day, day], [min_depth, max_depth], lw=1, color='black')

            legend_entry += 1

    print("  ASO comparison")
    print_hash(stats)


def legend_italic(ax, offset):
    legend = ax.legend(bbox_to_anchor=(offset, 0.5), loc='center right')

    for text in legend.get_texts():
        if text.get_text() in plot_data.keys():
            text.set_fontstyle('italic')
    

def new_max(current, new_value):
    if current < new_value:
        max_depth = math.ceil(new_value)
        if (max_depth - new_value) > 0.5:
            max_depth -= 0.5
        
        return max_depth
    return current

def plot_variable(snobal_var, snotel, title, label, save_figure=False):
    figure, axes = plt.subplots(3, 1, sharex=True, **figure_opts)
    plt.subplots_adjust(hspace=0)
    ax_index = 0
    max_depth = 0
    
    for site_name in plot_data.keys():
        ax = axes[ax_index]
        max_snobal = plot_data[site_name]['snobal'].coarsen(**COARSEN_OPTS).max()
        min_snobal = plot_data[site_name]['snobal'].coarsen(**COARSEN_OPTS).min()
        mean_snobal = plot_data[site_name]['snobal'].coarsen(**COARSEN_OPTS).mean()
        
        min_snobal[snobal_var].plot(ax=ax, c='slategrey', alpha=0.8, lw=1, ls='--')
        max_snobal[snobal_var].plot(ax=ax, c='slategrey', alpha=0.8, lw=1, ls='--')
        ax.fill_between(
            max_snobal.time, 
            min_snobal[snobal_var].data.flatten(), 
            max_snobal[snobal_var].data.flatten(),
            label='iSnobal',
            color=plot_data[site_name]['color_snobal'], 
            alpha=0.7,
            zorder=0,
        )
        ax.plot(
            max_snobal.time,
            plot_data[site_name]['snotel'][snotel],
            label='SNOTEL', 
            color=plot_data[site_name]['color_snotel'], 
            alpha=0.8, lw=1.5
        )
        
        print(site_name)    

        if title == 'Snow Depth':
            print_hash(depth_numbers(site_name, max_snobal, min_snobal))
            aso_scatter(ax, site_name, max_snobal, min_snobal, mean_snobal)
        elif title == 'SWE':
            print_hash(swe_numbers(site_name, max_snobal))

        ax.set_title(None)
        ax.set_ylabel(None)

        ax.xaxis.set_major_formatter(xTicks)
        ax.set_xlim([plot_range[0], plot_range[-1]])
        ax.set_xlabel(f'Water Year {plot_range[-1].year}')
        
        ax.legend(loc='upper left', borderaxespad=0.15, fontsize=8)
        at = AnchoredText(
            site_name, 
            prop=dict(size=10), 
            frameon=True, 
            loc='upper right', 
            pad=0.3, 
            borderpad=0.25,
        )
        at.patch.set_boxstyle("round", pad=0., rounding_size=0.2)
        at.patch.set(edgecolor='lightgrey')
        ax.add_artist(at)
        
        max_depth = new_max(max_depth, max_snobal[snobal_var].max())
        max_depth = new_max(max_depth, plot_data[site_name]['snotel'][snotel].max())
        
        if ax_index == 1:
            ax.set_ylabel(label)
        
        ax_index += 1
    
    for ax in axes:
        ax.set_ylim(top=max_depth)
        
        if title == 'Snow Depth':
            ax.set_yticks(np.arange(0, max_depth))
            ax.set_ylim(bottom=-0.2)
        elif title == 'SWE':
            ax.set_yticks(np.arange(0, max_depth, 300))
            ax.set_ylim(bottom=-50)

    if save_figure:
        filename = f'{year}_{title}_{site_name}'.replace(' ', '-')
        plt.savefig(f"{FIGURES_DIR}/{filename}.png")

In [ ]:
def melt_date_snobal(mean_snobal, site):
    after_max_swe = mean_snobal.sel(time=slice(f"{year}-03-01", f"{year}-07-30")).where(mean_snobal.time > plot_data[site]['max_swe_snotel'], drop=True)
    return after_max_swe.where(after_max_swe != 0, drop=True).time.max().values


def plot_depth_diff(site, ax1):
    mean_snobal = plot_data[site]['snobal']['thickness'].coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y'], drop=True)
    melt_date = melt_date_snobal(mean_snobal, site)
    
    depth_diff = plot_data[site]['snotel']['Depth(m)'] - mean_snobal.values
    
    pre_melt = depth_diff[depth_diff.index < plot_data[site]['max_swe_snotel']]
    post_melt = depth_diff[(depth_diff.index > plot_data[site]['max_swe_snotel']) & (depth_diff.index <= melt_date)]

    ax1.plot([], [], ' ', label=site)
    depth_diff.plot(ax=ax1, 
                    color=plot_data[site_name]['color_snotel'], 
                    label='Difference')
    ax1.axvline(
        x=post_melt.index[0], 
        ls='--', lw=1, 
        color=plot_data[site_name]['color_snobal'], 
        label=f'Start melt season'
    )
    
    print(f"{site}")
    print("  Pre-melt mean: {:.2f}".format(pre_melt.mean()))
    print("  Post     mean: {:.2f}".format(post_melt.mean()))

## Depth 

In [ ]:
plot_variable('thickness', 'Depth(m)', 'Snow Depth', 'Snow Depth (m)')

### SNOTEL - iSnobal 

In [ ]:
fig, (ax1) = plt.subplots(1, 1, **figure_opts)

for site_name in plot_data.keys():
    plot_depth_diff(site_name, ax1)

ax1.set_ylabel(r'$\Delta$ Snow Depth (m)')
ax1.xaxis.set_major_formatter(xTicks)
ax1.set_xlim([plot_range[0], plot_range[-1]])
ax1.set_ylim(-1.2, 1.0)
ax1.set_yticks(np.arange(-1.2, 1.2, 0.2))
ax1.set_xlabel(f'Water Year {plot_range[-1].year}')
legend_italic(ax1, 1.3)

## SWE

In [ ]:
plot_variable('specific_mass', 'SWE(mm)', 'SWE', 'SWE (mm)')

## Snow Density

In [ ]:
# plot_variable('snow_density', 'Density(kg/m3)', 'Snow Density', r'Snow Density ($kg/m^3$)')

## Temperature 

In [ ]:
# plot_variable('temp_surf', 'Air-T(C)', 'temp_lower', 'Temperature (C)')

In [ ]:
client.shutdown()